# ライブラリのインポート

In [3]:
import numpy as np
import utils as ut
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision.models as models
import multiprocessing

# Discriminator

In [4]:
class Discriminator(nn.Module):
    
    def __init__(self, item_num, user_num, emb_dim,
                           lamda, param=None, initdelta=0.05):
        super(Discriminator, self).__init__()
        self.item_num = item_num
        self.user_num = user_num
        self.emb_dim = emb_dim
        self.lamda = lamda
        self.param = param
        self.initdelta = initdelta
        self.d_params = []
        
        if self.param is None:
            self.user_embeddings = nn.Embedding(
                self.user_num, self.emb_dim)            
            self.item_embeddings = nn.Embedding(
                self.item_num, self.emb_dim)            
            self.item_bias = nn.Embedding(
                self.item_num, 1)
            
            # initialize
            torch.nn.init.uniform_(
                self.user_embeddings.weight, a=-initdelta, b=initdelta)
            torch.nn.init.uniform_(
                self.item_embeddings.weight, a=-initdelta, b=initdelta)
#             torch.nn.init.uniform_(
#                 self.item_bias.weight, a=initdelta, b=-initdelta)
            
    def forward(self, user, item, label):
        pre_logits = (self.user_embeddings(user) \
                              * self.item_embeddings(item)).squeeze().sum(1).view(-1,1) \
                                + self.item_bias(item).view(-1, 1)
        pre_loss =  F.binary_cross_entropy_with_logits(pre_logits, label)
        return pre_loss

    def all_rating(self, user):
        rating = torch.mm(self.user_embeddings(user).view(-1, 5),
                                               self.item_embeddings.weight.t()) + self.item_bias
        return all_rating

    def all_logits(self, user):
        logits = (self.user_embeddings(user) \
                              * self.item_embeddings.weight).sum(1) + self.item_bias
        return all_logits

    def get_reward(self, user, item):
        reward_logits = (self.user_embeddings(user) * self.item_embeddings(item)).squeeze().sum(1).view(-1,1) + self.item_bias(item).view(-1, 1)
        reward = 2 * (torch.sigmoid(reward_logits) - 0.5)
        return reward

# Generator

In [5]:
class Generator(nn.Module):
    
    def __init__(self, item_num, user_num, emb_dim,
                           lamda, param=None, initdelta=0.05, lr=0.05):
        super(Generator, self).__init__()
        self.item_num = item_num
        self.user_num = user_num
        self.emb_dim = emb_dim
        self.lamda = lamda
        self.param = param
        self.initdelta = initdelta
        self.lr = lr
        self.g_params = []
        
        import pickle
        with open("ml-100k/model_dns_ori.pkl", "rb") as f:
            param = pickle.load(f, encoding='latin1')
    
        if self.param is None:
            self.user_embeddings = nn.Embedding(
                self.user_num, self.emb_dim)
            self.item_embeddings = nn.Embedding(
                self.item_num, self.emb_dim)
            self.item_bias = torch.zeros(self.item_num)
            
        self.user_embeddings.weight = torch.nn.Parameter(torch.tensor(param[0]))
        self.item_embeddings.weight = torch.nn.Parameter(torch.tensor(param[1]))
        
            # initialize
#             torch.nn.init.uniform_(
#                 self.user_embeddings.weight, a=-initdelta, b=initdelta)
#             torch.nn.init.uniform_(
#                 self.item_embeddings.weight, a=-initdelta, b=initdelta)
        
    def forward(self, user, item, reward):
        softmax_score = F.softmax(self.all_logits(user).view(1, -1), -1)
        i_prob = torch.gather(softmax_score.view(-1), 0, item).clamp(min=1e-8)
        loss = - torch.mean(torch.log(i_prob) * reward) # \
#                    + self.lamda * (F.normalize(self.user_embeddings(user), p=2, dim=1) \
#                                              + F.normalize(self.item_embeddings(item), p=2, dim=1) \
#                                              + F.normalize(self.item_bias(item), p=2, dim=1))
        return loss

    def all_rating(self, user):
        rating = torch.mm(self.user_embeddings(user).view(-1, 5),
                                               self.item_embeddings.weight.t()) + self.item_bias
        return rating
    
    def all_logits(self, user):
        logits = (self.user_embeddings(user) \
                              * self.item_embeddings.weight).sum(1) + self.item_bias
        return logits

# 評価関数

In [6]:
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    return np.sum(r / np.log2(np.arange(2, r.size + 2)))


def ndcg_at_k(r, k):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

def simple_test_one_user(x):
    
    # import pdb; pdb.set_trace()
    rating = x[0]
    u = x[1]

    test_items = list(all_items - set(user_pos_train[u]))
    item_score = []
    for i in test_items:
        item_score.append((i, rating[i]))

    item_score = sorted(item_score, key=lambda x: x[1])
    item_score.reverse()
    item_sort = [x[0] for x in item_score]

    r = []
    for i in item_sort:
        if i in user_pos_test[u]:
            r.append(1)
        else:
            r.append(0)

    p_3 = np.mean(r[:3])
    p_5 = np.mean(r[:5])
    p_10 = np.mean(r[:10])
    
    ndcg_3 = ndcg_at_k(r, 3)
    ndcg_5 = ndcg_at_k(r, 5)
    ndcg_10 = ndcg_at_k(r, 10)

    return np.array([p_3, p_5, p_10, ndcg_3, ndcg_5, ndcg_10])

def simple_test(model):
    result = np.array([0.] * 6)
    pool = multiprocessing.Pool(cores)
    batch_size = 128
    test_users = list(user_pos_test.keys())
    test_user_num = len(test_users)
    index = 0
    
    while True:
        if index >= test_user_num:
            break
        user_batch = test_users[index:index + batch_size]
        index += batch_size
    
#         print(user_batch)
        user_batch_rating = model.all_rating(torch.tensor(user_batch))

        user_batch_rating = user_batch_rating.detach_().cpu().numpy()

        user_batch_rating_uid = zip(user_batch_rating, user_batch)
        batch_result = pool.map(simple_test_one_user, user_batch_rating_uid)
        for re in batch_result:
            result += re

    pool.close()
    ret = result / test_user_num
    ret = list(ret)
    return ret

# ハイパーパラメーター

In [7]:
EMB_DIM = 5
USER_NUM = 943
ITEM_NUM = 1683
BATCH_SIZE = 16
INIT_DELTA = 0.05

all_items = set(range(ITEM_NUM))
workdir = "ml-100k/"
DIS_TRAIN_FILE = workdir + "dis-train.txt"
cores = multiprocessing.cpu_count()

# ロードデータ

In [8]:
# positiveな要素だけを引っ張ってくる
user_pos_train = {}
with open(workdir + 'movielens-100k-train.txt')as fin:
    for line in fin:
        line = line.split()
        uid = int(line[0])
        iid = int(line[1])
        r = float(line[2])
        if r > 3.99:
            if uid in user_pos_train:
                user_pos_train[uid].append(iid)
            else:
                user_pos_train[uid] = [iid]

# testでnegativeな要素だけを引っ張ってくる                
user_pos_test = {}
with open(workdir + 'movielens-100k-test.txt')as fin:
    for line in fin:
        line = line.split()
        uid = int(line[0])
        iid = int(line[1])
        r = float(line[2])
        if r > 3.99:
            if uid in user_pos_test:
                user_pos_test[uid].append(iid)
            else:
                user_pos_test[uid] = [iid]

all_users = user_pos_train.keys()

# Generatorによるデータセレクト

In [9]:
def generate_for_d(model, filename):
    data = []
    for user in user_pos_train:
        pos = user_pos_train[user]

        rating = model.all_rating(torch.tensor(user))
        rating = rating.detach_().cpu().numpy()
        rating = np.array(rating) / 0.2  # Temperature
        exp_rating = np.exp(rating)
        prob = exp_rating / np.sum(exp_rating)

        neg = np.random.choice(np.arange(ITEM_NUM), size=len(pos), p=prob.reshape(-1,))
        for i in range(len(pos)):
            data.append(str(user) + '\t' + str(pos[i]) + '\t' + str(neg[i]))

    with open(filename, 'w')as fout:
        fout.write('\n'.join(data))

In [10]:
generator = Generator(
    ITEM_NUM, USER_NUM,EMB_DIM, lamda=0.0 / BATCH_SIZE,
    param=None, initdelta=INIT_DELTA)

discriminator = Discriminator(
    ITEM_NUM, USER_NUM,EMB_DIM, lamda=0.0 / BATCH_SIZE,
    param=None, initdelta=INIT_DELTA)

g_optimizer = torch.optim.SGD(
    generator.parameters(), lr=0.001, momentum=0.9)

d_optimizer = torch.optim.SGD(
    discriminator.parameters(), lr=0.001, momentum=0.9)                    

In [11]:
from torch import nn
from torch import autograd

generator.train()
discriminator.train()
for epoch in range(15):
    if epoch >= 0:
        for d_epoch in range(100):
            if d_epoch % 5 == 0:
                generate_for_d(generator, DIS_TRAIN_FILE)
                train_size = ut.file_len(DIS_TRAIN_FILE)
            index = 1
            while True:
                if index > train_size:
                    break
                if index + BATCH_SIZE <= train_size + 1:
                    users, items, labels = ut.get_batch_data(
                        DIS_TRAIN_FILE, index, BATCH_SIZE)
                else:
                    users, items, labels = ut.get_batch_data(
                        DIS_TRAIN_FILE, index, train_size - index + 1)
                    
                index += BATCH_SIZE
                users = torch.tensor(users).view(-1, 1)
                items = torch.tensor(items).view(-1, 1)
                labels = torch.tensor(labels).view(-1, 1)
    
                d_loss = discriminator(users, items, labels)
                d_optimizer.zero_grad()
                d_loss.backward()
                d_optimizer.step()
            print("\r[D Epoch %d/%d] [loss: %f]" %(d_epoch, 100, d_loss.item()))

        for g_epoch in range(50):
            for user in user_pos_train:
                sample_lambda = 0.2
                pos = user_pos_train[user]

                rating = generator.all_logits(torch.tensor(user))
                rating = rating.detach_().cpu().numpy()
                exp_rating = np.exp(rating)
                prob = exp_rating / np.sum(exp_rating)
                
                pn = (1 - sample_lambda) * prob
                pn[pos] += sample_lambda * 1.0 / len(pos)
                
                sample = np.random.choice(range(ITEM_NUM), 2*len(pos), p=pn)
                reward = discriminator.get_reward(torch.tensor([user] * 2 * len(pos)), torch.tensor(sample))
                reward = reward.detach_().cpu().numpy()
                reward = reward * prob[sample] / pn[sample]

                g_loss = generator(torch.tensor(user), torch.tensor(sample), torch.tensor(reward))
                g_optimizer.zero_grad()
                g_loss.backward()
                g_optimizer.step()
                
            print("\r[G Epoch %d/%d] [loss: %f]" %(g_epoch, 50, g_loss.item()))
            result = simple_test(generator)
            print("epoch ", epoch, "gen: ", result)
#             buf = '\t'.join([str(x) for x in result])
#             gen_log.write(str(epoch) + '\t' + buf + '\n')
#             gen_log.flush()

[D Epoch 0/100] [loss: 0.861321]
[D Epoch 1/100] [loss: 0.860576]
[D Epoch 2/100] [loss: 0.859837]
[D Epoch 3/100] [loss: 0.859101]
[D Epoch 4/100] [loss: 0.858370]
[D Epoch 5/100] [loss: 0.857644]
[D Epoch 6/100] [loss: 0.856921]
[D Epoch 7/100] [loss: 0.856204]
[D Epoch 8/100] [loss: 0.855491]
[D Epoch 9/100] [loss: 0.854781]
[D Epoch 10/100] [loss: 0.854076]
[D Epoch 11/100] [loss: 0.853376]
[D Epoch 12/100] [loss: 0.852679]
[D Epoch 13/100] [loss: 0.851986]
[D Epoch 14/100] [loss: 0.851298]
[D Epoch 15/100] [loss: 0.850614]
[D Epoch 16/100] [loss: 0.849934]
[D Epoch 17/100] [loss: 0.849258]
[D Epoch 18/100] [loss: 0.848586]
[D Epoch 19/100] [loss: 0.847917]
[D Epoch 20/100] [loss: 0.847254]
[D Epoch 21/100] [loss: 0.846593]
[D Epoch 22/100] [loss: 0.845937]
[D Epoch 23/100] [loss: 0.845286]
[D Epoch 24/100] [loss: 0.844638]
[D Epoch 25/100] [loss: 0.843994]
[D Epoch 26/100] [loss: 0.843354]
[D Epoch 27/100] [loss: 0.842718]
[D Epoch 28/100] [loss: 0.842085]
[D Epoch 29/100] [loss: 

[G Epoch 28/50] [loss: 0.280824]
epoch  0 gen:  [0.33187134502924009, 0.30614035087719282, 0.27149122807017512, 0.3434592720003079, 0.32644494241825239, 0.31290886054927891]
[G Epoch 29/50] [loss: 0.361999]
epoch  0 gen:  [0.33260233918128684, 0.30570175438596475, 0.27149122807017512, 0.34410857443663656, 0.3262735598854653, 0.31300375876306474]
[G Epoch 30/50] [loss: 0.456485]
epoch  0 gen:  [0.33260233918128684, 0.30570175438596475, 0.27105263157894699, 0.34410857443663656, 0.3262735598854653, 0.31273831198759394]
[G Epoch 31/50] [loss: 0.295187]
epoch  0 gen:  [0.33260233918128684, 0.30614035087719282, 0.27149122807017512, 0.34397383200691184, 0.32646390838418166, 0.31294053501000313]
[G Epoch 32/50] [loss: -0.055654]
epoch  0 gen:  [0.33260233918128684, 0.30570175438596475, 0.27127192982456105, 0.34397383200691178, 0.32617617795104048, 0.31278622440284981]
[G Epoch 33/50] [loss: 0.416605]
epoch  0 gen:  [0.33260233918128684, 0.30570175438596475, 0.27127192982456105, 0.3439738320069

[G Epoch 6/50] [loss: -0.422090]
epoch  1 gen:  [0.33187134502924009, 0.30438596491228059, 0.2710526315789471, 0.3434592720003079, 0.32532661558114928, 0.31264706868523251]
[G Epoch 7/50] [loss: -0.058768]
epoch  1 gen:  [0.33260233918128684, 0.30438596491228059, 0.2710526315789471, 0.34410857443663651, 0.32547555837696462, 0.31277791009066497]
[G Epoch 8/50] [loss: 0.331407]
epoch  1 gen:  [0.33260233918128684, 0.30482456140350866, 0.2710526315789471, 0.34410857443663651, 0.32573069391464449, 0.31276217511505983]
[G Epoch 9/50] [loss: 0.080864]
epoch  1 gen:  [0.33260233918128684, 0.30526315789473668, 0.2710526315789471, 0.34410857443663651, 0.32601842434778555, 0.31275559992813556]
[G Epoch 10/50] [loss: -0.410067]
epoch  1 gen:  [0.33260233918128684, 0.30526315789473668, 0.2710526315789471, 0.34410857443663651, 0.32601842434778555, 0.31275559992813556]
[G Epoch 11/50] [loss: 0.182193]
epoch  1 gen:  [0.33260233918128684, 0.30526315789473668, 0.2710526315789471, 0.34410857443663651, 

[D Epoch 15/100] [loss: 0.769051]
[D Epoch 16/100] [loss: 0.768817]
[D Epoch 17/100] [loss: 0.768585]
[D Epoch 18/100] [loss: 0.768354]
[D Epoch 19/100] [loss: 0.768124]
[D Epoch 20/100] [loss: 0.767895]
[D Epoch 21/100] [loss: 0.767667]
[D Epoch 22/100] [loss: 0.767441]
[D Epoch 23/100] [loss: 0.767215]
[D Epoch 24/100] [loss: 0.766990]
[D Epoch 25/100] [loss: 0.766767]
[D Epoch 26/100] [loss: 0.766544]
[D Epoch 27/100] [loss: 0.766323]
[D Epoch 28/100] [loss: 0.766102]
[D Epoch 29/100] [loss: 0.765883]
[D Epoch 30/100] [loss: 0.765664]
[D Epoch 31/100] [loss: 0.765447]
[D Epoch 32/100] [loss: 0.765230]
[D Epoch 33/100] [loss: 0.765015]
[D Epoch 34/100] [loss: 0.764800]
[D Epoch 35/100] [loss: 0.764587]
[D Epoch 36/100] [loss: 0.764374]
[D Epoch 37/100] [loss: 0.764163]
[D Epoch 38/100] [loss: 0.763952]
[D Epoch 39/100] [loss: 0.763742]
[D Epoch 40/100] [loss: 0.763534]
[D Epoch 41/100] [loss: 0.763326]
[D Epoch 42/100] [loss: 0.763119]
[D Epoch 43/100] [loss: 0.762913]
[D Epoch 44/10

[G Epoch 31/50] [loss: -0.424095]
epoch  2 gen:  [0.3340643274853804, 0.30614035087719282, 0.27192982456140319, 0.34540717930929388, 0.32693907228371899, 0.31425776413439721]
[G Epoch 32/50] [loss: -0.052214]
epoch  2 gen:  [0.3340643274853804, 0.30614035087719282, 0.27192982456140319, 0.34540717930929388, 0.32693907228371899, 0.31427585280694725]
[G Epoch 33/50] [loss: -0.341111]
epoch  2 gen:  [0.3340643274853804, 0.30614035087719282, 0.27192982456140319, 0.34540717930929388, 0.32693907228371899, 0.31427585280694725]
[G Epoch 34/50] [loss: -0.248846]
epoch  2 gen:  [0.3340643274853804, 0.30614035087719282, 0.27192982456140319, 0.34540717930929388, 0.32693907228371899, 0.31428281990888235]
[G Epoch 35/50] [loss: -0.186603]
epoch  2 gen:  [0.3340643274853804, 0.30570175438596481, 0.27192982456140319, 0.34540717930929388, 0.32665134185057793, 0.31425104925686836]
[G Epoch 36/50] [loss: 0.009240]
epoch  2 gen:  [0.33333333333333365, 0.30614035087719282, 0.27192982456140319, 0.34489261930

...

[D Epoch 81/100] [loss: 0.615695]
[D Epoch 82/100] [loss: 0.615334]
[D Epoch 83/100] [loss: 0.614973]
[D Epoch 84/100] [loss: 0.614612]
[D Epoch 85/100] [loss: 0.614250]
[D Epoch 86/100] [loss: 0.613887]
[D Epoch 87/100] [loss: 0.613524]
[D Epoch 88/100] [loss: 0.613160]
[D Epoch 89/100] [loss: 0.612795]
[D Epoch 90/100] [loss: 0.612430]
[D Epoch 91/100] [loss: 0.612064]
[D Epoch 92/100] [loss: 0.611698]
[D Epoch 93/100] [loss: 0.611331]
[D Epoch 94/100] [loss: 0.610963]
[D Epoch 95/100] [loss: 0.610595]
[D Epoch 96/100] [loss: 0.610226]
[D Epoch 97/100] [loss: 0.609856]
[D Epoch 98/100] [loss: 0.609486]
[D Epoch 99/100] [loss: 0.609115]
[G Epoch 0/50] [loss: -1.495908]
epoch  12 gen:  [0.33406432748538051, 0.30438596491228059, 0.27061403508771897, 0.34744711894649788, 0.32684430403749215, 0.31495146923909945]
[G Epoch 1/50] [loss: -1.059547]
epoch  12 gen:  [0.33552631578947412, 0.30438596491228059, 0.27061403508771897, 0.34861098138943047, 0.32701221279923681, 0.31508806584847132]
[G

[G Epoch 43/50] [loss: -1.697978]
epoch  12 gen:  [0.33625730994152087, 0.30482456140350866, 0.27127192982456111, 0.34861098138943042, 0.32701728749675829, 0.31519114576619905]
[G Epoch 44/50] [loss: -1.395586]
epoch  12 gen:  [0.33552631578947412, 0.30482456140350866, 0.27127192982456111, 0.34809642138282643, 0.32696572663536771, 0.31518190208205005]
[G Epoch 45/50] [loss: -1.169480]
epoch  12 gen:  [0.33552631578947412, 0.30482456140350866, 0.27127192982456111, 0.34809642138282643, 0.32696572663536771, 0.31516506571135172]
[G Epoch 46/50] [loss: -1.512090]
epoch  12 gen:  [0.33552631578947412, 0.30482456140350866, 0.27149122807017517, 0.34809642138282643, 0.32699832153082903, 0.31531277454989171]
[G Epoch 47/50] [loss: -1.539401]
epoch  12 gen:  [0.33552631578947412, 0.30482456140350866, 0.2710526315789471, 0.34809642138282643, 0.32699832153082903, 0.31503885478100324]
[G Epoch 48/50] [loss: -1.174887]
epoch  12 gen:  [0.33479532163742731, 0.30526315789473668, 0.27127192982456111, 0.

[G Epoch 21/50] [loss: -1.595291]
epoch  13 gen:  [0.33552631578947412, 0.30394736842105258, 0.2710526315789471, 0.34782693652337704, 0.32630414463132396, 0.31491919538966168]
[G Epoch 22/50] [loss: -1.147838]
epoch  13 gen:  [0.33552631578947412, 0.30394736842105258, 0.2710526315789471, 0.34809642138282643, 0.3264989085001736, 0.31503118439411559]
[G Epoch 23/50] [loss: -1.272978]
epoch  13 gen:  [0.33552631578947412, 0.30394736842105258, 0.2710526315789471, 0.34809642138282643, 0.32653150339563491, 0.31506928880904073]
[G Epoch 24/50] [loss: -1.090888]
epoch  13 gen:  [0.33552631578947412, 0.30394736842105258, 0.2710526315789471, 0.34796167895310171, 0.3264667163566714, 0.31502970334053904]
[G Epoch 25/50] [loss: -1.336622]
epoch  13 gen:  [0.33552631578947412, 0.30394736842105258, 0.2710526315789471, 0.34809642138282643, 0.32653150339563491, 0.31506096059343686]
[G Epoch 26/50] [loss: -1.206918]
epoch  13 gen:  [0.33552631578947412, 0.30438596491228059, 0.2710526315789471, 0.3480964

[D Epoch 91/100] [loss: 0.530860]
[D Epoch 92/100] [loss: 0.530437]
[D Epoch 93/100] [loss: 0.530015]
[D Epoch 94/100] [loss: 0.529592]
[D Epoch 95/100] [loss: 0.529170]
[D Epoch 96/100] [loss: 0.528747]
[D Epoch 97/100] [loss: 0.528325]
[D Epoch 98/100] [loss: 0.527903]
[D Epoch 99/100] [loss: 0.527481]
[G Epoch 0/50] [loss: -1.159644]
epoch  14 gen:  [0.33698830409356767, 0.30438596491228059, 0.27149122807017517, 0.34950535897291346, 0.32722945470258669, 0.31560173715114542]
[G Epoch 1/50] [loss: -1.060254]
epoch  14 gen:  [0.33625730994152092, 0.30438596491228059, 0.27149122807017517, 0.34885605653658486, 0.32708051190677129, 0.31551175073359256]
[G Epoch 2/50] [loss: -1.582360]
epoch  14 gen:  [0.33698830409356767, 0.30438596491228059, 0.27171052631578918, 0.34937061654318874, 0.32713207276816186, 0.31568332736846627]
[G Epoch 3/50] [loss: -1.447076]
epoch  14 gen:  [0.33698830409356767, 0.30394736842105252, 0.27127192982456111, 0.3497504341200679, 0.32711884659058899, 0.3155581273

[G Epoch 45/50] [loss: -1.187895]
epoch  14 gen:  [0.33698830409356773, 0.30614035087719282, 0.2710526315789471, 0.34975043412006795, 0.32859009365175579, 0.31553503937668648]
[G Epoch 46/50] [loss: -2.101188]
epoch  14 gen:  [0.33698830409356773, 0.30570175438596475, 0.27083333333333304, 0.34937061654318874, 0.32802785896304654, 0.31515589117267656]
[G Epoch 47/50] [loss: -0.752619]
epoch  14 gen:  [0.33698830409356773, 0.30570175438596475, 0.27083333333333304, 0.34937061654318874, 0.32799526406758522, 0.31510395795756635]
[G Epoch 48/50] [loss: -1.839454]
epoch  14 gen:  [0.33698830409356773, 0.30570175438596475, 0.2710526315789471, 0.34937061654318874, 0.32806045385850785, 0.31532647508898493]
[G Epoch 49/50] [loss: -1.450778]
epoch  14 gen:  [0.33698830409356773, 0.30526315789473668, 0.27083333333333304, 0.34937061654318874, 0.3278053183208281, 0.31517756629748911]


In [116]:
generator.user_embeddings.weight = torch.nn.Parameter(torch.tensor(param[0]))
generator.item_embeddings.weight = torch.nn.Parameter(torch.tensor(param[1]))

In [108]:
generator

Generator(
  (user_embeddings): Embedding(943, 5)
  (item_embeddings): Embedding(1683, 5)
  (item_bias): Embedding(1683, 1)
)